In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [4]:
%load_ext nb_black
%load_ext nb_mypy

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
The nb_mypy extension is already loaded. To reload it, use:
  %reload_ext nb_mypy


<IPython.core.display.Javascript object>

In [5]:
import nbimporter
from AIBaseClass import ChessAI
from Exercise07AI import Exercise07AI

<IPython.core.display.Javascript object>

# Aufgabe 08: Minimax mit Alpha-Beta-Pruning, Memoisierung, Progressive Deepening und 

Dieses Notebook erweitert den Minimax-Algorithmus mit Alpha-Beta-Pruning und Memoisierung um das Progressive Deepening.

In [12]:
import chess_custom as chess
from typing import Any


class Exercise08AI(Exercise07AI):
    """Chooses middle game moves using minimax algorithm, alpha-beta-pruning,
    memoization and progressive deepening and the quiescence search"""

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)

<IPython.core.display.Javascript object>

Die Funktion `get_next_middle_game_move` wurde um eine `for`-Schleife zur iterativen Tiefensuche erweitert.

In [9]:
class Exercise08AI(Exercise08AI):  # type: ignore
    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move:
        """Gets the best next move"""
        self.last_evaluation: int | None  # type annotation for mypy
        if self.is_king_endgame != self.check_king_endgame(board):
            self.last_evaluation += self.get_endgame_evaluation_change(board)
        # Calculate current evaluation
        if self.last_evaluation is None:  # type: ignore
            current_evaluation = self.full_evaluate(board)
        else:
            # Get current evaluation (after opponent move)
            current_evaluation = self.evaluate(board, self.last_evaluation)

        for depth in range(1, self.DEPTH + 1):
            # Call minimax and get best move
            _, best_move = self.minimax(board, depth, current_evaluation)

        # Reset local cache
        self.local_cache: dict[tuple, int] = {}

        # Debugging fail save
        assert best_move, f"""
        Best move is None with fen '{board.fen()}' at player {type(self).__name__}! 
        depth: {self.DEPTH}, last_eval: {self.last_evaluation}, current_evaluation: {current_evaluation},
        is_king_engame: {getattr(self, 'is_king_endgame', "N/A")}, move_stack: {board.move_stack}
        """
        # Update last evaluation (after player move)
        self.last_evaluation = current_evaluation + self.incremental_evaluate(
            board, best_move
        )
        return best_move

<IPython.core.display.Javascript object>

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
# Create player and board
unit_test_player = Exercise08AI(player_name="Ex08AI", search_depth=2)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
board

In [ ]:
# Test minimax
evaluation = unit_test_player.full_evaluate(board)  # Get current evaluation
mm_evaluation, mm_move = unit_test_player.minimax(board, 2, evaluation)
assert mm_evaluation == 355, "Minimax evaluation does not match expected value!"
assert mm_move.uci() == 'd4c3', "Minimax move does not match expected value!"
mm_evaluation, mm_move

In [ ]:
# Test next move function
move = unit_test_player.get_next_middle_game_move(board)
assert move.uci() == 'd4c3', "Next move does not match expected value!"
unit_test_player.last_evaluation = None
move

## Temporärer Bereich

Der folgende Bereich dient zum temporären Debuggen und kann nicht-funktionierenden Code enthalten. Dieser Bereich wird vor der Abgabe entfernt.